In [ ]:
#Q1-1)

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [78]:
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [79]:
#train과 test set으로 나누어 MNIST data 불러오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [93]:
#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할것!)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [96]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)
linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

In [97]:
bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)

In [98]:
# #xavier initialization을 이용하여 각 layer의 weight 초기화
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.1705, -0.0645,  0.1800,  0.1802,  0.1703,  0.1303, -0.1934,  0.1475,
          0.2066,  0.2283, -0.0985, -0.0142, -0.1564,  0.1342, -0.1575, -0.1771,
         -0.2079,  0.0791,  0.0404,  0.1153, -0.1480,  0.1202,  0.2274,  0.2032,
         -0.0060, -0.0896, -0.1560, -0.1778,  0.1009,  0.1711, -0.0967,  0.2232,
         -0.0501,  0.2000,  0.0812,  0.1326, -0.2145, -0.0174,  0.1273,  0.1139,
          0.1220,  0.2320, -0.0646, -0.2181,  0.0025,  0.0364,  0.1440,  0.2151,
         -0.1932, -0.0106,  0.1983,  0.1032,  0.0936,  0.0068,  0.1349,  0.2009,
         -0.0169, -0.1541, -0.0118,  0.0229, -0.2052, -0.1700,  0.0066,  0.0144,
          0.1112,  0.1943, -0.0945, -0.0184, -0.1992, -0.1985,  0.0043,  0.2204,
         -0.1416,  0.1688,  0.1050,  0.2257,  0.0783,  0.2126, -0.1226, -0.1962,
          0.1485,  0.2098, -0.1196,  0.0221, -0.2192,  0.1278,  0.1387, -0.0675,
         -0.0740, -0.0945,  0.1729,  0.0038, -0.1371,  0.0677,  0.2317, -0.2116,
      

In [99]:
# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-BatchNormalization Layer - ReLU- DropOut)
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3)

In [100]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

In [104]:
#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것)

for epoch in range(training_epochs):
    model.train() #모델의 train 설정
    avg_cost = 0
    total_batch = len(data_loader)
    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여loss를 최적화하는 코드

    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.335887700
Epoch: 0002 cost = 0.313184172
Epoch: 0003 cost = 0.295466810
Epoch: 0004 cost = 0.282583892
Epoch: 0005 cost = 0.281316608
Epoch: 0006 cost = 0.280670136
Epoch: 0007 cost = 0.269507557
Epoch: 0008 cost = 0.259180784
Epoch: 0009 cost = 0.263238132
Epoch: 0010 cost = 0.249530151
Epoch: 0011 cost = 0.247522503
Epoch: 0012 cost = 0.239278078
Epoch: 0013 cost = 0.234324142
Epoch: 0014 cost = 0.229313299
Epoch: 0015 cost = 0.230627790
Learning finished


In [105]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model.eval()
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9416999816894531


In [107]:
##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드
r = random.randint(0, len(mnist_test)-1)
X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
Y_single_data = mnist_test.test_labels[r:r + 1]
print('Label: ', Y_single_data.item())
single_prediction = model(X_single_data)
print('Prediction: ', torch.argmax(single_prediction, 1).item())

Label:  2
Prediction:  2


In [ ]:
#Q2

In [108]:
# 각 Layer의 Hidden node 수 : 1st Layer (784,200), 2nd Layer(200,150),3rd Layer(150,10)
linear4 = torch.nn.Linear(784, 200, bias=True)
linear5 = torch.nn.Linear(200, 150, bias=True)
linear6 = torch.nn.Linear(150, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

In [110]:
bn3 = torch.nn.BatchNorm1d(200)
bn4 = torch.nn.BatchNorm1d(150)

In [111]:
# #xavier initialization을 이용하여 각 layer의 weight 초기화
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-1.4672e-01,  1.5654e-01, -1.6575e-01,  1.4880e-01,  1.8373e-02,
          1.3713e-01, -8.7996e-02, -1.3954e-01, -6.6759e-02,  1.2603e-01,
          8.9636e-03, -7.9346e-03,  4.9936e-02, -2.0084e-01, -4.1163e-03,
         -4.0050e-02, -2.2770e-01,  1.1679e-01,  1.4914e-02,  1.0584e-01,
          2.2078e-01,  1.5680e-01, -4.1690e-03,  1.2568e-01,  1.2713e-01,
         -1.1062e-01,  6.4293e-02, -1.3294e-01,  1.3039e-01,  1.5983e-01,
          1.4998e-01, -1.7174e-01, -1.0874e-01, -5.2801e-02, -1.9846e-01,
          7.0162e-02,  6.8782e-02, -2.2479e-01, -3.0747e-02, -9.4651e-02,
          1.0536e-01, -7.3873e-02, -1.2459e-01, -1.7649e-02, -1.2496e-01,
          2.1045e-01, -5.7543e-02,  1.2448e-01,  1.9021e-01,  2.1779e-01,
         -9.9571e-02, -1.0864e-01,  1.6517e-01, -1.3926e-01,  1.3346e-01,
          1.0061e-01, -2.6361e-02, -2.0429e-01, -2.2332e-02, -7.4129e-03,
         -9.1403e-02, -2.2731e-01, -7.7730e-02, -9.2895e-02,  1.0127e-01,
         -6.0260

In [113]:
# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-BatchNormalization Layer - ReLU- DropOut)
model_2 = torch.nn.Sequential(linear4, bn1, relu, dropout,
                            linear5, bn2, relu, dropout,
                            linear6)

In [114]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

In [116]:
#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것)

for epoch in range(training_epochs):
    model_2.train() #모델의 train 설정
    avg_cost = 0
    total_batch = len(data_loader)
    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여loss를 최적화하는 코드

    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        hypothesis = model_2(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.187100127
Epoch: 0002 cost = 0.188789889
Epoch: 0003 cost = 0.185400680
Epoch: 0004 cost = 0.177967817
Epoch: 0005 cost = 0.183117822
Epoch: 0006 cost = 0.185920298
Epoch: 0007 cost = 0.175034031
Epoch: 0008 cost = 0.176049247
Epoch: 0009 cost = 0.164263591
Epoch: 0010 cost = 0.168677479
Epoch: 0011 cost = 0.165325537
Epoch: 0012 cost = 0.170857623
Epoch: 0013 cost = 0.163866669
Epoch: 0014 cost = 0.155931011
Epoch: 0015 cost = 0.148930907
Learning finished


In [117]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model_2.eval()
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels

    prediction = model_2(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9043999910354614


In [118]:
##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드
r = random.randint(0, len(mnist_test)-1)
X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
Y_single_data = mnist_test.test_labels[r:r + 1]
print('Label: ', Y_single_data.item())
single_prediction = model_2(X_single_data)
print('Prediction: ', torch.argmax(single_prediction, 1).item())

Label:  5
Prediction:  5
